# Aspen Benchmark

Tutorial for the Aspen Benchmark application. Aspen Benchmark connects to an Aspen Plus simulation which then can be optimized by a BO algorithm from BoFire and BoTorch.

Make sure to close running Aspen Plus instances in the Task Manager to not cover licenses if not needed.

### Imports

In [ ]:
import pandas as pd
from bofire.benchmarks.aspen_benchmark import Aspen_benchmark
from bofire.domain.constraint import LinearInequalityConstraint
from functools import partial
from bofire.utils.multiobjective import compute_hypervolume, get_pareto_front
from bofire.domain.domain import Domain
from bofire.benchmarks.benchmark import run
from bofire.strategies.random import RandomStrategy
from bofire.strategies.botorch.qehvi import BoTorchQnehviStrategy
from bofire.models.torch_models import BotorchModels
from bofire.domain.objective import MaximizeObjective, MinimizeObjective
from bofire.samplers import PolytopeSampler
from bofire.domain.features import (CategoricalInput, ContinuousInput,
                                    ContinuousOutput,
                                    OutputFeatures)
from benchmarks.heat_pump_case_study_V5 import Heat_Pump_Case_Study_V5
hp = Heat_Pump_Case_Study_V5()

## Manual Setup of the optimization domain

In [ ]:
# Define the input features that are supposed to be given to Aspen before each simulation run.
# These are the parameters that are suggested by the optimizer.

input_features = [
    ContinuousInput(
        key="THX1",
        lower_bound=57,
        upper_bound=65
    ),
    ContinuousInput(
        key="TW1",
        lower_bound=50,
        upper_bound=60
    ),
    ContinuousInput(
        key="TW2",
        lower_bound=55,
        upper_bound=66.5
    ),
    ContinuousInput(
        key="TW4",
        lower_bound=55,
        upper_bound=66.5
    ),
    ContinuousInput(
        key="DTVAP",
        lower_bound=-20,
        upper_bound=-3
    ),
    ContinuousInput(
        key="TCOND",
        lower_bound=105,
        upper_bound=125
    ),
    ContinuousInput(
        key="DTSG",
        lower_bound=5,
        upper_bound=20
    ),
    ContinuousInput(
        key="TCRYST",
        lower_bound=60,
        upper_bound=65
    ),
    CategoricalInput(
        key="WF",
        categories=["R1233", "N-BUT", "ISOBUT"],
        # values=[["R1233"], ["N-BUT"], ["ISOBUT"]]
    ),
]

# Define the ouput values of the Aspen simulation that are supposed to be optimized.
# Each values needs a name "key" and information about whether it should be minmized "MinimizeObjective" or maximized "MaximizeObjective".
output_features = [
    ContinuousOutput(
        key="QIN",
        objective=MaximizeObjective(w=1.0)
    ),
    ContinuousOutput(
        key="PEL",
        objective=MinimizeObjective(w=1.0)
    ),
    ContinuousOutput(
        key="CAPEX",
        objective=MinimizeObjective(w=1.0)
    ),
]


# Define constraints that describe relationships between input values and thus limit the input domain.
# E.g. x1- 2*x2 <= 0
# Linear inequality constraints need to be manipulated into the form "something <= right-hand-side".
# The involved variable names need to be passed as "features" within a list.
# Coefficients are passed in the same order in another list, while the right-hand-side is passed as a constant.
constraints = [
    LinearInequalityConstraint(
        features=["THX1", "TW1"],
        coefficients=[-1, 1],
        rhs=-2,
    ),
    LinearInequalityConstraint(
        features=["TW1", "TCRYST"],
        coefficients=[1, -1],
        rhs=-8,
    ),
    LinearInequalityConstraint(
        features=["TW2", "TW1"],
        coefficients=[-1, 1],
        rhs=-3,
    ),
    LinearInequalityConstraint(
        features=["TCRYST", "TW4"],
        coefficients=[-1, 1],
        rhs=-2,
    ),
    LinearInequalityConstraint(
        features=["TW4", "TW1"],
        coefficients=[-1, 1],
        rhs=-3,
    ),
    LinearInequalityConstraint(
        features=["TCOND", "DTSG"],
        coefficients=[-1, 1],
        rhs=-100,
    ),
]

# Create the domain object
domain = Domain(
    input_features=input_features,
    output_features=output_features,
    constraints=constraints
)

## Setup of the Variable Paths to Aspen

The transfer of variables between Python and Aspen Plus is based on the Python interface of Aspen. For more info see https://kitchingroup.cheme.cmu.edu/blog/2013/06/14/Running-Aspen-via-Python/. Each simulation variable of Aspen Plus can be accessed manually through the variable explorer in the program "Customize -> Variable Explorer". Similarly, Python can read and write values from and into the variable tree. Therefore, the variable paths through that tree need to be provided.

In [ ]:
# Store the paths to each variable within a dictionary with the varaible names as the keys and the paths as the values.

paths = {
    "THX1": "\\Data\\Blocks\\HX-01-1\\Input\\VALUE",
    "TW1": "\\\Data\\Streams\\W1\\Input\\TEMP\\MIXED",
    "TW2": "\\Data\\Blocks\\DUM-01\\Input\\TEMP",
    "TW4": "\\Data\\Blocks\\DUM-02\\Input\\TEMP",
    "DTVAP": "\\Data\\Blocks\\DUM-10\\Input\\DELT",
    "TCOND": "\\Data\\Blocks\\DUM-11\\Input\\TEMP",
    "DTSG": "\\Data\\Flowsheeting Options\\Calculator\\CA-01\Input\\FVN_INIT_VAL\\DTSG",
    "TCRYST": "\\Data\\Flowsheeting Options\\Calculator\\CA-06\Input\\FVN_INIT_VAL\\TCRYST",
    "WF": "\\Data\\Flowsheeting Options\\Calculator\\CA-07\Input\\FVN_INIT_VAL\\WF",
    "QIN": "\\Data\\Flowsheeting Options\\Calculator\\OBJ-01\Output\\WRITE_VAL\\2",
    "PEL": "\\Data\\Flowsheeting Options\\Calculator\\OBJ-02\Output\\WRITE_VAL\\9",
    "CAPEX": "\\Data\\Flowsheeting Options\\Calculator\\OBJ-03\Output\\WRITE_VAL\\22"
}

## Aspen Readability

Depending on the implementation of the simulation in Aspen Plus itself, certain input values can differ between Aspen and BoFire.
Categorical inputs for example need to be set as discrete integer values in Aspen whereas BoFire uses strings for each category. To translate into the Aspen-readable version, a conversion function is needed. This is not necessary for continuous inputs.

In [ ]:
# The conversion function is passed to Aspen_benchmark and will be called before new values are going to be passed to the simulation.
# It needs the "domain" and the input values "candidates" as inputs and needs to return the input dataframe containing the translated columns that are aspen-readable.

def conversion_function(domain: Domain, candidates: pd.DataFrame) -> pd.DataFrame:
    # Iterate through input features to find the inputs, that need to be translated.
            for feature in domain.inputs.features:
                # Translate "CategoricalInputs"
                if feature.type == "CategoricalInput":
                    translated_values = []
                    for elem in candidates[feature.key]:
                        if elem == "R1233":
                            value = 1
                        elif elem == "N-BUT":
                            value = 2
                        else:
                            value = 3
                        translated_values.append(value)
                # Add elif for other input types that require a translation.

                    candidates[feature.key] = translated_values
            
            return candidates

## Initialization

In [ ]:
# Provide the filename to where the Aspen simulation file is located.
# Make sure it is a .apwz file.
filename = "aspen_simulations/heat_pump_aspen_sim_V5.apwz"
aspen_benchmark = Aspen_benchmark(
    filename=filename,
    domain=domain,
    paths=paths,
    translate_into_aspen_readable=conversion_function
)

## Sampling and Hypervolume Functions

The sampling fuction generates random input values according the the constraints that are the start points for the optimizer.

To assess the bayesian optimization algorithm, a hypervolume function is needed. The hypervolume function returns the current hypervolume after each run which tells the optimizer the amount of improvement. The hypervolume is computed from a reference point that needs to be derived from a first random run.

In [ ]:
def sample(domain):
    sampler = PolytopeSampler(domain=domain)
    sampled = sampler.ask(15)
    return sampled

In [ ]:
ref_point = {
    "QIN": -20,
    "PEL": 30,
    "CAPEX" :50
}

def hypervolume(domain: Domain):
    assert domain.experiments is not None
    pareto_points = get_pareto_front(
        domain=domain,
        experiments=domain.experiments,
        output_feature_keys=domain.outputs.get_keys()
    )
    hypervolume = compute_hypervolume(
                    domain=domain,
                    optimal_experiments=pareto_points,
                    ref_point=ref_point
                )
    return (hypervolume)

### Generate Initial Values

As Aspen Plus simulations take time, it is preferred to generate a set of initial values in advance in case the strategy stops.

In [8]:
x_initials = sample(domain=domain)
initials = aspen_benchmark.f(candidates=x_initials)
initials = pd.concat([x_initials, initials], axis=1)
aspen_benchmark.domain.add_experiments(initials)
initials

,DTSG,DTVAP,TCOND,TCRYST,THX1,TW1,TW2,TW4,WF,CAPEX,valid_CAPEX,PEL,valid_PEL,QIN,valid_QIN
0,18.472520,-19.100071,118.850776,63.595212,57.552661,51.186898,60.913280,56.529062,ISOBUT,43.453610,1,26.098742,1,-31.291158,1
1,14.776622,-19.598761,124.171483,64.699359,57.179683,50.199942,60.605731,55.211359,N-BUT,43.186927,1,25.951932,1,-31.102559,1
2,12.725963,-4.125543,116.166564,64.029775,58.867158,52.157423,57.649255,58.341157,N-BUT,34.365679,1,18.349122,1,-30.721235,1
3,7.939984,-14.387707,120.551216,64.943720,60.927911,52.012236,56.199943,56.468543,R1233,37.884113,1,19.898957,1,-29.491663,1
4,8.589781,-5.739696,123.327012,64.891115,58.249162,55.323921,61.525135,59.945177,R1233,34.157888,1,17.550963,1,-30.712509,1
5,13.250104,-4.457855,117.094151,62.164859,58.512170,50.049839,60.357842,59.399525,ISOBUT,36.139893,1,20.798496,1,-31.360764,1
6,12.667969,-8.351805,120.922572,60.334855,59.582448,50.289771,64.358245,55.538442,ISOBUT,37.631273,1,23.112448,1,-31.432174,1
7,9.792672,-14.258642,120.750571,64.102470,60.191408,51.073429,60.888817,58.612289,R1233,39.642198,1,20.657311,1,-30.120519,1
8,12.266466,-7.031157,124.428151,62.699990,64.311197,50.522937,63.717545,58.042668,R1233,31.391998,1,16.427023,1,-26.736373,1
9,13.044612,-8.624023,124.501264,64.109664,63.307181,50.667096,65.393891,58.811390,N-BUT,31.570694,1,18.721591,1,-27.754701,1


## Run Random Strategy

In [9]:
random_results = run(
    aspen_benchmark,
    strategy_factory=RandomStrategy,
    n_iterations=1,
    metric=hypervolume,
    n_runs=1,
    n_procs=1,
)

run 00 with current best 0.000: 100%|██████████| 1/1 [00:08<00:00,  8.09s/it]


## Run QNEHVI Strategy

### Automatic Run

An automatic run can easily be done if only continuous inputs are present.

In [ ]:
results = run(
    aspen_benchmark,
    strategy_factory=partial(BoTorchQnehviStrategy, ref_point=ref_point),
    n_iterations=15,
    metric=hypervolume,
    n_runs=1,
    n_procs=1,
)

### Manual setup

In case of a categorical input following model specifications work faster and more efficient.

In [10]:
from bofire.models.gps.gps import SingleTaskGPModel
from bofire.utils.enum import CategoricalMethodEnum

model_specs = BotorchModels(
    models=[
        SingleTaskGPModel(
            input_features=aspen_benchmark.domain.inputs,
            output_features=OutputFeatures(features=[aspen_benchmark.domain.outputs[i]])
        )
    for i in range(len(domain.output_features))]
)

results = run(
    benchmark=aspen_benchmark,
    strategy_factory=partial(
        BoTorchQnehviStrategy,
        ref_point=ref_point,
        categorical_method = CategoricalMethodEnum.EXHAUSTIVE,
        model_specs = model_specs,
    ),
    n_iterations=2,
    metric=hypervolume, 
    initial_sampler=sample,
    n_procs=1,
    n_runs=1,
)

  0%|          | 0/2 [00:00<?, ?it/s]C:\Users\J33463\OneDrive - Evonik Industries AG\sandbox\botorch\botorch\optim\initializers.py:208: BadInitialCandidatesWarning: Unable to find non-zero acquisition function values - initial conditions are being selected randomly.
  warnings.warn(
C:\Users\J33463\OneDrive - Evonik Industries AG\sandbox\botorch\botorch\optim\initializers.py:208: BadInitialCandidatesWarning: Unable to find non-zero acquisition function values - initial conditions are being selected randomly.
  warnings.warn(


## Performance Plot

In [ ]:
import plotly.express as px

random_results_df = random_results[0][0].domain.experiments
random_results_df["strategy"] = "RANDOM"  # type: ignore
results_df = results[0][0].domain.experiments.iloc[:,15:-1]  # type: ignore
results_df["strategy"] = "QNEHVI"  # type: ignore
initials["strategy"] = "INITIALS (random)"
ref_df = pd.DataFrame(ref_point, index=[0])
ref_df["strategy"] = "REF POINT"
df_to_plot = pd.concat([initials, random_results_df, results_df, ref_df], axis=0)  # type: ignore
df_to_plot.reset_index(inplace=True, drop=True)

df_to_plot

In [ ]:
px.scatter_matrix(
    results_df,
    dimensions=["PEL", "CAPEX", "QIN"],
    color="strategy",
    width=1200,
    height=900
)